In [1]:
import numpy as np
import pandas as pd
import glob 
import os
import datetime
pd.options.display.max_columns=50

In [1]:
results = glob.glob('./results/performances/www21-result-*')   # prelim-all-include-semi/
list1, list2 = zip(*sorted(zip([os.stat(result).st_size for result in results], results)))

NameError: name 'glob' is not defined

In [3]:
results = glob.glob('./results/query_indices/*/')

In [4]:
results

['./results/query_indices/1602592798.118/']

In [5]:

list1, list2 = zip(*sorted(zip([os.stat(result).st_size for result in results], results))) 
df = pd.read_csv(list2[-15], skiprows = 1)

IndexError: tuple index out of range

### Result of Hybrid Experiments

In [ ]:
import csv

run_id_saver = [('hybrid', 1602420493.392)]
results = {}
num_weeks = 5 # ToDo: to be automatically determined

for strategy, run_id in run_id_saver:
    results[run_id] = {}
    for week in range(num_weeks):
        filename = f'results/query_indices/{run_id}/{run_id}-{strategy}-5.0-scratch-week-{week}.csv'
        with open(filename, "r") as f:
            reader = csv.reader(f, delimiter=",")
            expid = next(reader)[1]
            dataset = next(reader)[1]
            episode = next(reader)[1]
            start_day = next(reader)[1]
            end_day = next(reader)[1]
            
            start_day = datetime.date(int(start_day[:4]), int(start_day[5:7]), int(start_day[8:10])).strftime('%y-%m-%d')
            end_day = datetime.date(int(end_day[:4]), int(end_day[5:7]), int(end_day[8:10])).strftime('%y-%m-%d')
            
            if week == 0:
                if dataset == 'real-n':
                    df = pd.read_csv('data/ndata.csv')
                elif dataset == 'real-t':
                    df = pd.read_csv('data/tdata.csv')
                elif dataset == 'real-m':
                    df = pd.read_csv('data/mdata.csv')
                elif dataset == 'synthetic':
                    df = pd.read_csv('data/synthetic-imports-declarations.csv')
                
            alldata = df[(df['sgd.date'] < end_day) & (df['sgd.date'] >= start_day)].loc[:, ['illicit', 'revenue']]
            
            while True:
                try:
                    indices = next(reader)
                    samp = indices[0]
                    indices = indices[1:]
                    indices = list(map(int, indices))

                    if week == 0:
                        results[run_id][f'{samp}-pre'] = []
                        results[run_id][f'{samp}-rec'] = []
                        results[run_id][f'{samp}-rev'] = []

                    chosen = df.iloc[indices].loc[:, ['illicit', 'revenue']]
                    # Recall and revenue
                    if chosen.empty:
                        pre = rec = rev = 0
                    else:
                        pre = sum(chosen['illicit'])/chosen['illicit'].count()
                        rec = sum(chosen['illicit'])/sum(alldata['illicit'])
                        rev = sum(chosen['revenue'])/sum(alldata['revenue'])

                    print(f'Week {week}, subsampler {samp}: Precision {round(pre, 4)} Recall {round(rec, 4)}, Revenue {round(rev, 4)}')
                    results[run_id][f'{samp}-pre'].append(pre)
                    results[run_id][f'{samp}-rec'].append(rec)
                    results[run_id][f'{samp}-rev'].append(rev)
                
                except StopIteration:
                    break

In [ ]:
run_id = 1602420493.392
pd.DataFrame.from_dict(results[run_id])

### Result to find novel frauds

In [ ]:
run_id_saver = [('hybrid', 1602420493.392)]

novelty = {}
num_weeks = 5 # ToDo: to be automatically determined
old_IID = set()

for strategy, run_id in run_id_saver:
    novelty[run_id] = {}
    for week in range(num_weeks):
        filename = f'results/query_indices/{run_id}/{run_id}-{strategy}-5.0-scratch-week-{week}.csv'
        with open(filename, "r") as f:
            reader = csv.reader(f, delimiter=",")
            expid = next(reader)[1]
            dataset = next(reader)[1]
            episode = next(reader)[1]
            start_day = next(reader)[1]
            end_day = next(reader)[1]
            
            start_day = datetime.date(int(start_day[:4]), int(start_day[5:7]), int(start_day[8:10])).strftime('%y-%m-%d')
            end_day = datetime.date(int(end_day[:4]), int(end_day[5:7]), int(end_day[8:10])).strftime('%y-%m-%d')
            
            if week == 0:
                if dataset == 'real-n':
                    df = pd.read_csv('data/ndata.csv')
                elif dataset == 'real-t':
                    df = pd.read_csv('data/tdata.csv')
                elif dataset == 'real-m':
                    df = pd.read_csv('data/mdata.csv')
                elif dataset == 'synthetic':
                    df = pd.read_csv('data/synthetic-imports-declarations.csv')
                
            alldata = df[(df['sgd.date'] < end_day) & (df['sgd.date'] >= start_day)].loc[:, ['illicit', 'revenue', 'importer.id']]
            alldata = alldata[~alldata['importer.id'].isin(old_IID)]
            
            if alldata.empty:
                continue
            
            while True:
                try:
                    indices = next(reader)
                    samp = indices[0]
                    indices = indices[1:]
                    indices = list(map(int, indices))

                    if week == 0:
                        novelty[run_id][f'{samp}-pre'] = []
                        novelty[run_id][f'{samp}-rec'] = []
                        novelty[run_id][f'{samp}-rev'] = []

                    chosen = df.iloc[indices]
                    chosen = chosen[~chosen['importer.id'].isin(old_IID)]
                    
                    # Recall and revenue
                    if chosen.empty:
                        pre = rec = rev = 0
                    else:
                        pre = sum(chosen['illicit'])/chosen['illicit'].count()
                        rec = sum(chosen['illicit'])/sum(alldata['illicit'])
                        rev = sum(chosen['revenue'])/sum(alldata['revenue'])

                    print(f'Week {week}, subsampler {samp}: Precision {round(pre, 4)} Recall {round(rec, 4)}, Revenue {round(rev, 4)}')
                    novelty[run_id][f'{samp}-pre'].append(pre)
                    novelty[run_id][f'{samp}-rec'].append(rec)
                    novelty[run_id][f'{samp}-rev'].append(rev)
                
                    old_IID = old_IID.union(set(alldata['importer.id'].values))
                    
                except StopIteration:
                    break


In [ ]:
run_id = 1602420493.392
pd.DataFrame.from_dict(novelty[run_id])